In [1]:
#import os
#os.environ['ESMFMKFILE'] = '/home/malasawedah/.conda/envs/test2/lib/esmf.mk'

# Now you can import the ESMF library
#import ESMF


In [2]:
#!conda install -c conda-forge xesmf -y #>= 0.7.1

In [3]:
import datetime
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
#import xvec
import shapely
from shapely import Point, box
import esmpy
import xagg as xa
import rioxarray

# supress warnings
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

Found esmf.mk at: /home/cgoehner/.conda/envs/xesmf_env/lib/esmf.mk


In [4]:
xa

<module 'xagg' from '/home/cgoehner/xagg/xagg/__init__.py'>

In [5]:
#!pip install dask

In [6]:
aoi = gpd.read_file('/mnt/CEPH_PROJECTS/ScaleAgData/03_Ancillary_Data/LAFIS_zonal_stats_prato_stabile_1000.shp')
grid5 = pd.read_csv('/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/polygons_grid_5')
#chunks = {'t': 1, 'x': 100, 'y': 100}
#s1 = xr.open_dataset(f'/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/latest/GRD_luca/asc/grid5/S1_grid5_1/openEO.nc', chunks= chunks)
s1 = xr.open_dataset(f'/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/latest/GRD_luca/asc/grid5/S1_grid5_1/openEO.nc')

In [7]:
s1 = s1.fillna(0)

In [8]:
s1.rio.write_crs("EPSG:32632", inplace=True)

<xarray.Dataset>
Dimensions:  (t: 12, y: 2282, x: 3764)
Coordinates:
    crs      int64 0
  * t        (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-02 2021-04-03
  * x        (x) float64 6.478e+05 6.478e+05 6.478e+05 ... 6.854e+05 6.854e+05
  * y        (y) float64 5.17e+06 5.17e+06 5.17e+06 ... 5.147e+06 5.147e+06
Data variables:
    VV       (t, y, x) float32 0.0 0.0 0.0 0.0 ... 0.06706 0.1505 0.06671 0.1619
    VH       (t, y, x) float32 0.0 0.0 0.0 0.0 ... 0.04054 0.0371 0.02428
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.15.1a1
    description:  
    title:

In [9]:
s1_wgs84 = s1.rio.reproject("EPSG:4326")

In [10]:
s1_wgs84

<xarray.Dataset>
Dimensions:  (x: 4159, y: 1788, t: 12)
Coordinates:
  * x        (x) float64 10.92 10.92 10.92 10.92 ... 11.42 11.42 11.42 11.42
  * y        (y) float64 46.67 46.67 46.67 46.67 ... 46.46 46.46 46.46 46.46
  * t        (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-02 2021-04-03
    crs      int64 0
Data variables:
    VV       (t, y, x) float32 3.403e+38 3.403e+38 ... 3.403e+38 3.403e+38
    VH       (t, y, x) float32 3.403e+38 3.403e+38 ... 3.403e+38 3.403e+38
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.15.1a1
    description:  
    title:

In [11]:
grid5['geometry'] = grid5['geometry'].apply(lambda x: shapely.wkt.loads(x))
gdf_grid5 = gpd.GeoDataFrame(grid5, geometry='geometry')
gdf_grid5.crs = 'EPSG:4326'
gdf_grid5.head(3)

,Unnamed: 0,ID,FID,CUAA,CODE,DESCR_IT,DESCR_DE,area,dem_mean,dem_median,slo_mean,slo_median,asp_mean,asp_median,north_mean,north_medi,dem_class,geometry
0,0,2894152,PRGMFR75E13F132Z-1688030,PRGMFR75E13F132Z,AP2,Prato stabile,Wiese (Dauerwiese),56,1301.867221,1301.705200,17.897289,17.806307,85.131834,84.665092,1.000000,1.0,1300.0,"POLYGON ((11.02662 46.46839, 11.02669 46.46830..."
1,1,2347633,RNRGTH80H18F132V-59,RNRGTH80H18F132V,AP2,Prato stabile,Wiese (Dauerwiese),31147,1302.440111,1302.553589,22.939459,22.398680,145.304115,153.931000,0.043887,0.0,1300.0,"POLYGON ((10.94554 46.65611, 10.94556 46.65604..."
2,2,2970168,WSSCRS85L19A952F-2158250,WSSCRS85L19A952F,AP2,Prato stabile,Wiese (Dauerwiese),842,1318.083754,1317.940063,12.269650,12.262682,228.961556,228.782425,0.000000,0.0,1300.0,"POLYGON ((11.13220 46.49439, 11.13249 46.49438..."


In [12]:
#chunks = {'t': 1, 'x': 100, 'y': 100}
#s1 = xr.open_dataset(f'/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/latest/GRD_luca/asc/grid5/S1_grid5_1/openEO.nc', chunks= chunks)
#s1_wgs84 = s1_wgs84.chunk({'t': 1, 'x': 50, 'y': 50})


In [13]:
%%time
weightmap = xa.pixel_overlaps(s1_wgs84, gdf_grid5)

creating polygons for each pixel...
lat/lon bounds not found in dataset; they will be created.
calculating overlaps between pixels and output polygons...
success!
CPU times: user 2min 27s, sys: 5.5 s, total: 2min 33s
Wall time: 2min 33s


In [17]:
ds = s1_wgs84
wm = weightmap
# impl='for_loop'
silent=False

In [15]:
# # Make sure pixel_overlaps was correctly run if using dot product
# if (impl=='dot_product') and (wm.overlap_da is None):
#     raise ValueError("no 'overlap_da' was found in the `wm` input - since you're using the dot product implementation, "+
#                      "make sure to run `pixel_overlaps()` with `impl='dot_product'` to avoid this error.")

# # Turn into dataset if dataarray
# if type(ds)==xr.core.dataarray.DataArray:
#     if ds.name is None:
#         warnings.warn('An unnamed xr.DataArray was inputted instead of a xr.Dataset; the output variable will be "var"')
#         ds = ds.to_dataset(name='var')
#     else:
#         ds = ds.to_dataset()


# # Run ds through fix_ds (to fix lat/lon names, lon coords)
# ds = xa.fix_ds(ds)

# # Stack 
# ds = ds.stack(loc=('lat','lon'))

In [18]:
ds

<xarray.Dataset>
Dimensions:  (x: 4159, y: 1788, t: 12)
Coordinates:
  * x        (x) float64 10.92 10.92 10.92 10.92 ... 11.42 11.42 11.42 11.42
  * y        (y) float64 46.67 46.67 46.67 46.67 ... 46.46 46.46 46.46 46.46
  * t        (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-02 2021-04-03
    crs      int64 0
Data variables:
    VV       (t, y, x) float32 3.403e+38 3.403e+38 ... 3.403e+38 3.403e+38
    VH       (t, y, x) float32 3.403e+38 3.403e+38 ... 3.403e+38 3.403e+38
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.15.1a1
    description:  
    title:

In [19]:
# ds = xa.subset_find(ds,wm.source_grid)

In [18]:
# if type(wm.weights) == pd.core.series.Series:
#     weights = np.array([float(k) for k in wm.weights])
# else:
#     if wm.weights != 'nowghts':
#         warnings.warn('wm.weights is: \n '+print(wm.weights)+
#                         ', \n which is not a supported weight vector (in a pandas series) '+
#                         'or "nowghts" as a string. Assuming no weights are included...')
#     if impl=='dot_product':
#         weights = np.ones((len(wm.overlap_da['loc'])))
#     elif impl=='for_loop':
#         weights = np.ones((len(wm.source_grid['lat'])))

In [20]:
xa.aggregate

<function xagg.core.aggregate(ds, wm, impl='for_loop', stat='mean', skipna=True, silent=False)>

In [22]:
xa.aggregate(ds, wm, stat = "min").to_dataset()

adjusting grid... (this may happen because only a subset of pixels were used for aggregation for efficiency - i.e. [subset_bbox=True] in xa.pixel_overlaps())
grid adjustment successful
aggregating VV by min...
aggregating VH by min...
all variables aggregated to polygons!


<xarray.Dataset>
Dimensions:     (poly_idx: 101, t: 12)
Coordinates:
  * poly_idx    (poly_idx) int64 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
  * t           (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-03
Data variables: (12/19)
    Unnamed: 0  (poly_idx) int64 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
    ID          (poly_idx) int64 2894152 2347633 2970168 ... 2060657 2614883
    FID         (poly_idx) object 'PRGMFR75E13F132Z-1688030' ... 'NTRSFR65C25...
    CUAA        (poly_idx) object 'PRGMFR75E13F132Z' ... 'NTRSFR65C25I431Z'
    CODE        (poly_idx) object 'AP2' 'AP2' 'AP2' 'AP2' ... 'AP2' 'AP2' 'AP2'
    DESCR_IT    (poly_idx) object 'Prato stabile' ... 'Prato stabile'
    ...          ...
    asp_median  (poly_idx) float64 84.67 153.9 228.8 139.9 ... 198.7 60.5 158.1
    north_mean  (poly_idx) float64 1.0 0.04389 0.0 0.0 ... 0.0 0.009091 1.0 0.0
    north_medi  (poly_idx) float64 1.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 1.0 0.0
    dem_class   (poly_idx) float64 1.3e+03 1.3e+03 1.3e+03 ... 600.0 1.2e+03
    VV_min      (poly_idx) object [[array([0.        , 0.00121075, 0.        ...
    VH_min      (poly_idx) object [[array([0.00000000e+00, 6.70609670e-05, 0....